In [1]:
from pipeline import ClaimifyPipeline
from llm_client import LLMClient
import polars as pl

data = pl.read_parquet('../data/texts_with_metadata.parquet')

# print(data.head())
print(data["actual_lead_text"][0:3])

shape: (3,)
Series: 'actual_lead_text' [str]
[
	"Polisen i Jämtland ryckte ut m…
	"Socialdemokraterna har under l…
	"Den man som anhölls misstänkt …
]


In [3]:
llm_client = LLMClient()

first_text = data["actual_lead_text"][0]

pipeline = ClaimifyPipeline(llm_client)
result = pipeline.run(first_text)

2025-11-26 12:58:02,479 - claimify.pipeline - INFO - Structured outputs enabled for improved reliability
2025-11-26 12:58:02,480 - claimify.pipeline - INFO - Processing 2 sentences
2025-11-26 12:58:02,480 - claimify.pipeline - INFO - Processing sentence 1/2: Polisen i Jämtland ryckte ut med flera patruller efter ett samtal om en skadad person....
2025-11-26 12:58:02,481 - claimify.llm.vllm - INFO - === STRUCTURED LLM CALL #1 - STAGE: SELECTION ===
2025-11-26 12:58:02,481 - claimify.llm.vllm - INFO - Provider: vllm, Model: Qwen/Qwen3-8B
2025-11-26 12:58:02,482 - claimify.llm.vllm - INFO - Response Model: SelectionResponse
2025-11-26 12:58:02,482 - claimify.llm.vllm - INFO - System Prompt (6881 chars): You are an assistant to a fact-checker.
2025-11-26 12:58:02,483 - claimify.llm.vllm - INFO - User Prompt (319 chars): Question:
The user did not provide a question.

Excerpt:
Polisen i Jämtland ryckte ut med flera patruller efter ett samtal om en skadad person.
Brottsplatsen är utomhus och

2025-11-26 12:58:10,001 - claimify.llm.vllm - ERROR - Structured call #1 failed after 7.52s: Error during structured LLM API call: 'ChatCompletionMessage' object has no attribute 'parsed'
2025-11-26 12:58:10,002 - claimify.llm.vllm - ERROR - === END STRUCTURED CALL #1 (ERROR) ===

2025-11-26 12:58:10,002 - claimify.pipeline - INFO - SELECTION: Sentence error
2025-11-26 12:58:10,003 - claimify.pipeline - INFO - Processing sentence 2/2: Brottsplatsen är utomhus och en person har gripits i samband med utredningen....
2025-11-26 12:58:10,003 - claimify.llm.vllm - INFO - === STRUCTURED LLM CALL #2 - STAGE: SELECTION ===
2025-11-26 12:58:10,004 - claimify.llm.vllm - INFO - Provider: vllm, Model: Qwen/Qwen3-8B
2025-11-26 12:58:10,004 - claimify.llm.vllm - INFO - Response Model: SelectionResponse
2025-11-26 12:58:10,005 - claimify.llm.vllm - INFO - System Prompt (6881 chars): You are an assistant to a fact-checker.
2025-11-26 12:58:10,006 - claimify.llm.vllm - INFO - User Prompt (310 chars): Q

In [ ]:
from pydantic import BaseModel
from openai import OpenAI

class Claim(BaseModel):
    reasoning: str
    claim: str
    evidence: str

system_prompt = """
You are a helpful assistant that extracts claims from a text. Let reasoning capture your
full thought process and the later fields capture your final answer.
"""

user_prompt = """
Paris is the worst city in the world. When I went there I lost my wallet.
"""

response_model = Claim

client = OpenAI(base_url="http://localhost:8000/v1", api_key="")

response = client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=[{"role": "user", "content": user_prompt}],
    extra_body={
        "guided_json": response_model.model_json_schema(),
    },
)
print(response)

ChatCompletion(id='chatcmpl-197816238f834d6687f383614142d9db', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="<think>\nOkay, the user says Paris is the worst city in the world because they lost their wallet there. First, I need to acknowledge their frustration. Losing a wallet is definitely a hassle, so I should express empathy. But I should also be careful not to take their statement too literally. Paris is a popular tourist destination, so maybe they had a bad experience, but it's not accurate to say it's the worst city overall.\n\nI should consider possible reasons why they lost their wallet. Maybe they were in a crowded area, or they didn't take precautions. It's important to offer practical advice, like checking with local authorities or using tracking apps. Also, maybe suggest ways to prevent this in the future, like keeping valuables secure.\n\nI should also balance the response by mentioning that while Paris has its challeng

In [13]:
print(response.choices[0].message.content)
# print(response.choices[0].message.reasoning)

<think>
Okay, the user says Paris is the worst city in the world because they lost their wallet there. Let me break this down. First, I need to acknowledge their frustration. Losing a wallet can be really stressful, so validating their feelings is important. But I should also consider that their experience might be an isolated incident. Paris is a major tourist destination, so there are many people there, which could increase the chances of losing something. 

I should mention that while Paris has its challenges, like crowded places or pickpocketing, it's also a beautiful city with a lot to offer. Maybe suggest some tips for staying safe, like keeping valuables secure and being aware of surroundings. It's also good to remind them that not everyone has the same experience, so it's not fair to generalize the entire city. I should keep the tone empathetic but also balanced, offering practical advice without dismissing their experience. Let me make sure not to come off as defensive but rat